In [1]:
from ipywidgets import widgets
from pandas import DataFrame
from optimisation import property_results
from IPython.display import display

In [2]:
def f(filler, FR, stabiliser, DINP, LDH, sphFiller):
    phr = [100, filler, FR, stabiliser, DINP, LDH, sphFiller]
    ingredients = ['PVC', 'filler', 'FR', 'stabiliser', 'DINP', 'LDH', 'spherical_filler']
    
    mass_frac = [i/sum(phr) for i in phr]
    mass_frac_per = [round(100*i, 2) for i in mass_frac]
    
    df_form = DataFrame([phr, mass_frac_per], index=['phr', 'Mass Frac. %'], columns=ingredients).T
    
    df_pr = property_results(mass_frac, [], [], use_style=False)
    df_pr = df_pr[['max', 'min', 'value_unscaled']].round(2)
    
    show_these = ['rheomix stability_time_min',
                  'rheomix final_deg_time_min',
                  'tensile E_t_MPa',
                  'tensile epsilon_break_%',
                  'tensile sigma_break_MPa',
                  'ConeCal MARHE_kW_m2',
                  'ConeCal peak_HRR_kWpm2',
                  'ConeCal tot_smokeprod_m2',
                 ]
    
    df_pr = df_pr.loc[show_these]
    
    display(df_pr, df_form)

w = widgets.interact(f,
                 filler=(0, 70, 0.1),
                 FR=(0, 20, 0.1),
                 stabiliser=(2, 10, 0.1),
                 DINP=(20, 70, 0.1),
                 LDH=(0, 15, 0.1),
                 sphFiller=(0, 20, 0.1))
                 

,max,min,value_unscaled
name,,,
rheomix stability_time_min,141.40,8.50,39.65
rheomix final_deg_time_min,346.90,12.60,88.55
tensile E_t_MPa,1917.68,23.64,117.12
tensile epsilon_break_%,114.77,9.82,53.46
tensile sigma_break_MPa,29.51,10.61,15.91
ConeCal MARHE_kW_m2,179.16,12.10,118.77
ConeCal peak_HRR_kWpm2,242.76,60.00,166.61
ConeCal tot_smokeprod_m2,18.12,3.68,12.36


,phr,Mass Frac. %
PVC,100.0,46.60
filler,37.6,17.52
FR,10.0,4.66
stabiliser,5.7,2.66
DINP,44.4,20.69
LDH,7.0,3.26
spherical_filler,9.9,4.61
